In [13]:
import time
import string
import re
import numpy as np
import pandas as pd
import json
import os
import math

In [54]:
sample = "and mmmth tc pica plug    type r tcs at 9mm type k at mm and mmmth tc pica plug    type r tcs at 9mm type k at mm and mmmth tc pica plug  type k tc at mmmth tc pica plug  type k tc at mmmth tc pica plug  type k tc at mmmth tc pica plug  type k tc at mmmth tc pica plug  type k tc at mmmth9 tc pica plug  type k tc at mmmth tc pica plug  type k tc at mmmth tc pica plug  type k tc at mmmpb low pressure transducer  variable reluctance transducer with external rtd for temperaturemtb tc slav plug   type k tcs at mm mmmtb tc slav plug  type k tc at mmmtb tc slav plug  type k tc at mmmtb tc slav plug  type k tc at mmmtb tc slav plug  type k tc at mmmtb tc slav plug  type k tc at mmmtb heatflux sensor   schmidtboelter with surface tcmtb heatflux sensor   schmidtboelter with surface tcmtb9 radiometer   sapphire window over schmidtboelter with surface tcstagnation and turbulent heatingstagnation pressurebackshell pressure and heatingsymmetric acreageheatingbackshell radiationrtdp prioritiesmars  entry descent  landing edlmedli operatesm cameras operatemedli sensor support electronics sse measure"
re.sub(r"(\b[a-z]{2}\b|\b[a-z]{1}\b)", "",sample)

'and mmmth  pica plug    type  tcs  9mm type    and mmmth  pica plug    type  tcs  9mm type    and mmmth  pica plug  type    mmmth  pica plug  type    mmmth  pica plug  type    mmmth  pica plug  type    mmmth  pica plug  type    mmmth9  pica plug  type    mmmth  pica plug  type    mmmth  pica plug  type    mmmpb low pressure transducer  variable reluctance transducer with external rtd for temperaturemtb  slav plug   type  tcs   mmmtb  slav plug  type    mmmtb  slav plug  type    mmmtb  slav plug  type    mmmtb  slav plug  type    mmmtb  slav plug  type    mmmtb heatflux sensor   schmidtboelter with surface tcmtb heatflux sensor   schmidtboelter with surface tcmtb9 radiometer   sapphire window over schmidtboelter with surface tcstagnation and turbulent heatingstagnation pressurebackshell pressure and heatingsymmetric acreageheatingbackshell radiationrtdp prioritiesmars  entry descent  landing edlmedli operatesm cameras operatemedli sensor support electronics sse measure'

In [60]:
def clean(text):
    # text = re.sub("[^-9A-Za-z ]", "" , text)
    # text = re.sub(r"(\b[a-z]{2}\b|\b[a-z]{1}\b)", "", text)
    # test = re.sub(r"\s{2,}", " ", text)
    # text = "".join([i.lower() for i in text if i not in string.punctuation])
    return text

In [63]:
filenames = os.listdir("data/Aerothermodynamics")

texts = {}

for filename in filenames[:10]:
    with open(f"data/Aerothermodynamics/{filename}") as f:
        text = f.read()
        texts[filename.split(".")[0]] = clean(text)

In [65]:
texts["20210025518"]

'powerpoint presentationnational aeronautics and space administrationmars entry instrumentation flight data and mars  entry environmentsaiaa scitech conference mars  edl performance and instrumentation atmosphere aerodynamics and trajectory afmtpjanuary  todd white milad mahzari chun tang josh monk jose santosnasa ameschris karlgaardama incorporated  nasa langleyhannah alpertama incorporated  nasa ameshenry wright chris kuhlnasa langleybackground  edl instrumentation planetary entry probe missions are complex costly and infrequent it  important  the entry descent and landing edl and planetary exploration community  understand the entry environments performance  the entry vehicle and atmospheric density and winds flight instrumentation provides  valuable dataset  understand the flown environment beyond  binary  worked   failed areas  design conservatism accuracy and limitations  our statetheart predictive simulations and experiments this data can  used for improved reliability thermal p